# PCA on Covariates Simulations Notebook

In [2]:
# Import objects from the setup file
import os
import sys
sys.path.append(os.path.expanduser('~/repo/ECMA-31330-Project/Source'))
from ME_Setup import *

# Packages
import numpy as np
import pandas as pd
import seaborn as sns
from pca import pca
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FactorAnalysis
import math
from tqdm import tqdm

In [3]:
# Supressing Output
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

**My main changes and what I want people to review in the code block below:**
    
1) I added an IV section

2) I added a new thing where we take half of the mismeasured covariate values and then use them as the exponent to raise e to (e.g. z4 = e**z4)

3) i got rid of the 1,000 value for beta1 and beta2

4) i got rid of the p = 10 value

**Comments on those changes:**

1) i'm curious if i did this right

2) i did this because of bonhomme's feedback about where PCA might outperform taking the simple avg. we've gotten some tentative signs that PCA does do better than the avg when you stretch out he observations

3) , 4) it seemed like we didn't really need those values

In [4]:
# Data frame to store output
output = pd.DataFrame()

# 2,000 observations
N = 2000

# Loop over combinations of: betas, covariances between variables, p numbers of parameters
for beta1 in [0.1,1,10]:
    for beta2 in [0.1,1,10]:
        for covariance in [-0.9,-0.5,0,0.5,0.9]:
            for p in [5,20,50]:
                # Select only the scenarios we actually want to run by default - betas of 1, covariances of 0.5, p of 5... require three of these conditions to be satisfied to run.
                counter = 0
                if beta1 == 1:
                    counter += 1
                if beta2 == 1:
                    counter += 1
                if covariance == 0.5:
                    counter += 1
                if p == 5:
                    counter+=1
                if counter >= 3:
                    # Run with and without transformations
                    for exp_of_var in ['yes','no']:
                        # 1000 simulations
                        for k in tqdm(range(1)):
                            # Initialize Lists to store coef values for all five methods and the true coef
                            pca_coef = []
                            mismeasured_coef = []
                            mismeasured_allvar_coef = []
                            mismeasured_avg_coef = []
                            iv_coef = []
                            true_val_coef =[]

                            # Create variables
                            vars_mean = [0,0,0]
                            vars_cov = np.array([[1,covariance,0],
                                                 [covariance,1,0],
                                                 [0,0,1]])
                            # Producing 3 variables: x for the variable of interest, the true Z covariate, the random error
                            vars_ = pd.DataFrame(np.random.multivariate_normal(vars_mean, vars_cov, N), columns = ['x','true_z','u'])
                            vars_['y'] = beta1 * vars_['x'] + beta2 * vars_['true_z'] + vars_['u']

                            # Create measurement errors for each of the p measurements of the covariates- mean zero and variance one
                            errors_mean = np.zeros(p)
                            errors_cov = np.zeros((p,p))
                            for i in range(p):
                                for j in range(p):
                                    if i == j:
                                        errors_cov[i,j] = 1

                            errors = np.random.multivariate_normal(errors_mean, errors_cov, N)
                            # Column labels for Z variables (covariates variables mismeasured)
                            z_vars = []
                            for i in range(p):
                                z_vars.append('z'+str(i+1))
                            # Add errors to the true_z to get mismeasured values
                            mismeasured_z = pd.DataFrame(errors, columns = z_vars)
                            for i in mismeasured_z.columns:
                                mismeasured_z[i] = mismeasured_z[i] + vars_['true_z']

                            # Take e to the power of the values for half of the measurements if log_of_var is true
                            if exp_of_var == 'yes':
                                mismeasured_z.iloc[:,int(len(mismeasured_z.columns)/2):] =np.exp(mismeasured_z.iloc[:,int(len(mismeasured_z.columns)/2):])
                                
                            # Do feature scaling (normalize to mean 0 and variance 1) for the mismeasured z
                            # Note that x and y are already normalized by construction
                            scaled_mismeasured_z = mismeasured_z.copy()
                            for i in mismeasured_z.columns:
                                scaled_mismeasured_z[i] = (mismeasured_z[i] - mismeasured_z[i].mean()) / mismeasured_z[i].std()

                            # Suppress output
                            with suppress_stdout():
                                # Use PCA on the mismeasured values
                                pca_model = pca()
                                pca_results = pca_model.fit_transform(scaled_mismeasured_z)
                                pca_z = pca_results['PC']['PC1']

                            # NOTE: in non-pca cases, no need to rescale or normalize since mismeasured variables and x and y have mean 0 and sd 1

                            # Average mismeasured variables:
                            vars_['avg_mismeasured_z'] = mismeasured_z[z_vars].mean(axis=1)

                            # Add relevant variables to vars_ dataframe
                            vars_[mismeasured_z.columns] = mismeasured_z
                            vars_['pca_z'] = pca_z

                            # Single mismeasured covariate results
                            model_mismeasured = sm.OLS(vars_['y'],vars_[['x','z1']])
                            results_mismeasured = model_mismeasured.fit()
                            mismeasured_coef.append(results_mismeasured.params[0])

                            # All Variables Mismeasured Results
                            # Create full list of items to include in regression
                            tot_vars = ['x']
                            tot_vars.extend(z_vars)
                            model_mismeasured_allvar = sm.OLS(vars_['y'],vars_[tot_vars])
                            results_mismeasured_allvar = model_mismeasured_allvar.fit()
                            mismeasured_allvar_coef.append(results_mismeasured_allvar.params[0])

                            # Average Mismeasured Variables Results
                            model_mismeasured_avg = sm.OLS(vars_['y'],vars_[['x','avg_mismeasured_z']])
                            results_mismeasured_avg = model_mismeasured_avg.fit()
                            mismeasured_avg_coef.append(results_mismeasured_avg.params[0])

                            # PCA Results
                            model_pca = sm.OLS(vars_['y'],vars_[['x','pca_z']])
                            results_pca = model_pca.fit()
                            pca_coef.append(results_pca.params[0])

                            # Instrumental Variables Results
                            # Instrument z1 on the other items in the mismeasured df
                            z_string = mismeasured_z.columns[1]
                            for z in mismeasured_z.columns[2:]:
                                z_string = z_string + ' + ' + z
                            # Create the predicted value of z1
                            vars_['pred_z1'] = smf.ols("z1 ~ " + z_string, data = mismeasured_z).fit().predict()
                            iv_results = smf.ols("y ~ x + pred_z1 -1", data = vars_).fit()
                            iv_coef.append(iv_results.params[0])

                            # True Results
                            model_true = sm.OLS(vars_['y'],vars_[['x','true_z']])
                            results_true = model_true.fit()
                            true_val_coef.append(results_true.params[0])

                            # Output Findings
                            new_output = pd.DataFrame()
                            new_output['mismeasured_coef'] = mismeasured_coef
                            new_output['mismeasured_allvar_coef'] = mismeasured_allvar_coef
                            new_output['mismeasured_avg_coef'] = mismeasured_avg_coef
                            new_output['pca_coef'] = pca_coef
                            new_output['true_val_coef'] = true_val_coef
                            new_output['iv_coef'] = iv_coef
                            new_output['covariance'] = vars_cov[0][1]
                            new_output['beta1'] = beta1
                            new_output['beta2'] = beta2
                            new_output['p'] = p
                            new_output['exp_of_var'] = exp_of_var
                            output = output.append(new_output)

output

100%|██████████| 1/1 [00:00<00:00,  2.24it/s]


,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,covariance,beta1,beta2,p,exp_of_var
0,0.369439,0.259652,0.443448,0.266809,0.074569,0.314139,0.5,0.1,1.0,5,yes
0,0.381021,0.218270,0.220204,0.219955,0.112098,0.247333,0.5,0.1,1.0,5,no
0,1.037412,1.031412,1.043361,1.033029,1.008602,1.040223,0.5,1.0,0.1,5,yes
0,1.047652,1.032653,1.029586,1.030001,1.020066,1.030305,0.5,1.0,0.1,5,no
0,0.254457,0.394852,0.161883,0.328149,0.993651,0.322783,-0.9,1.0,1.0,5,yes
0,0.296391,0.539387,0.540122,0.538471,1.017593,0.484506,-0.9,1.0,1.0,5,no
0,0.718394,0.822276,0.640263,0.806271,1.006469,0.769591,-0.5,1.0,1.0,5,yes
0,0.729930,0.909727,0.910428,0.909792,1.025991,0.889803,-0.5,1.0,1.0,5,no
0,1.015138,1.014898,1.025816,1.033058,1.018260,1.026033,0.0,1.0,1.0,5,yes
0,1.021665,1.004850,1.005135,1.005376,0.985193,0.996373,0.0,1.0,1.0,5,no


In [5]:
output

,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,covariance,beta1,beta2,p,exp_of_var
0,0.369439,0.259652,0.443448,0.266809,0.074569,0.314139,0.5,0.1,1.0,5,yes
0,0.381021,0.218270,0.220204,0.219955,0.112098,0.247333,0.5,0.1,1.0,5,no
0,1.037412,1.031412,1.043361,1.033029,1.008602,1.040223,0.5,1.0,0.1,5,yes
0,1.047652,1.032653,1.029586,1.030001,1.020066,1.030305,0.5,1.0,0.1,5,no
0,0.254457,0.394852,0.161883,0.328149,0.993651,0.322783,-0.9,1.0,1.0,5,yes
0,0.296391,0.539387,0.540122,0.538471,1.017593,0.484506,-0.9,1.0,1.0,5,no
0,0.718394,0.822276,0.640263,0.806271,1.006469,0.769591,-0.5,1.0,1.0,5,yes
0,0.729930,0.909727,0.910428,0.909792,1.025991,0.889803,-0.5,1.0,1.0,5,no
0,1.015138,1.014898,1.025816,1.033058,1.018260,1.026033,0.0,1.0,1.0,5,yes
0,1.021665,1.004850,1.005135,1.005376,0.985193,0.996373,0.0,1.0,1.0,5,no


In [6]:
#output.to_csv('C://Users//paulo//Documents//Econometrics and ML//Final Project//n_2000_results.csv')

In [7]:
pd.set_option('display.max_rows', 10)

In [8]:
output['pca_ape'] = (output['pca_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_coef_ape'] = (output['mismeasured_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_allvar_coef_ape'] = (output['mismeasured_allvar_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_avg_coef_ape'] = (output['mismeasured_avg_coef'] - output['beta1']).abs()/output['beta1']
output['iv_coef_ape'] = (output['iv_coef'] - output['beta1']).abs()/output['beta1']

In [9]:
output.groupby(['covariance','beta1','beta2','p','exp_of_var']).mean().reset_index()

,covariance,beta1,beta2,p,exp_of_var,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,no,0.296391,0.539387,0.540122,0.538471,1.017593,0.484506,0.461529,0.703609,0.460613,0.459878,0.515494
1,-0.9,1.0,1.0,5,yes,0.254457,0.394852,0.161883,0.328149,0.993651,0.322783,0.671851,0.745543,0.605148,0.838117,0.677217
2,-0.5,1.0,1.0,5,no,0.729930,0.909727,0.910428,0.909792,1.025991,0.889803,0.090208,0.270070,0.090273,0.089572,0.110197
3,-0.5,1.0,1.0,5,yes,0.718394,0.822276,0.640263,0.806271,1.006469,0.769591,0.193729,0.281606,0.177724,0.359737,0.230409
4,0.0,1.0,1.0,5,no,1.021665,1.004850,1.005135,1.005376,0.985193,0.996373,0.005376,0.021665,0.004850,0.005135,0.003627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,0.5,1.0,10.0,5,yes,3.835763,2.736777,4.889809,3.033415,0.955749,3.386664,2.033415,2.835763,1.736777,3.889809,2.386664
18,0.5,10.0,1.0,5,no,10.284815,10.123299,10.123861,10.123623,10.025852,10.141373,0.012362,0.028481,0.012330,0.012386,0.014137
19,0.5,10.0,1.0,5,yes,10.257260,10.148153,10.348358,10.168153,9.979385,10.205653,0.016815,0.025726,0.014815,0.034836,0.020565
20,0.9,1.0,1.0,5,no,1.753470,1.513187,1.512977,1.512574,1.075158,1.565119,0.512574,0.753470,0.513187,0.512977,0.565119


In [10]:
output.groupby(['covariance','beta1','beta2','p','exp_of_var']).std().reset_index()

,covariance,beta1,beta2,p,exp_of_var,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.9,1.0,1.0,5,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.5,1.0,1.0,5,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.5,1.0,1.0,5,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,1.0,1.0,5,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,0.5,1.0,10.0,5,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,0.5,10.0,1.0,5,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,0.5,10.0,1.0,5,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,0.9,1.0,1.0,5,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,1.0,5,1.239877,1.130403,1.305751,1.144760,0.956871,1.172444,0.144760,0.239877,0.130403,0.305751,0.172444
1,0.5,1.0,1.0,20,1.264090,1.054896,1.296471,1.083604,1.014924,1.067577,0.083604,0.264090,0.054896,0.296471,0.067577
2,0.5,1.0,1.0,50,1.299301,1.043388,1.283434,1.067714,1.016150,1.056073,0.067714,0.299301,0.043388,0.283434,0.056073


In [12]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,1.0,1.0,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.5,1.0,1.0,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
def std_parenth_padder(type_col, col_to_pad):
    if type_col == 'std':
        return('(' + col_to_pad + ')')
    else:
        return(col_to_pad)

def format_level_0(type_col, col_to_format):
    if type_col == 'std':
        return('')
    else:
        return(col_to_format)

In [14]:
import regex as re

# Pass this function a pandas dataframe of which to create the LaTeX file
# dataframe is the df
# filename is the save location
# index tells if you want the index saved to the file or not
def Minimal_Latex(dataframe, filename, index = False):

    # Open file
    with open(filename, "w") as f:
        
        # Start from the pandas
        corrected_table = dataframe.to_latex(header = False, index = index)

        # Strip all material that pandas inserts- the tabular environment and toprule and bottomrule
        corrected_table = re.sub(r'\\begin.+', '', corrected_table)
        corrected_table = re.sub(r'\\toprule', '', corrected_table)
        corrected_table = re.sub(r'\\bottomrule', '', corrected_table)
        corrected_table = re.sub(r'\\end.+', '', corrected_table)

        # Write to file and skip blank lines
        f.write(corrected_table.strip())

In [15]:
coef_names_mapper = {'mismeasured_coef':'Single Measurement', 'mismeasured_allvar_coef':'All Measurements', 'pca_coef':'PCA', 'mismeasured_avg_coef':'Average of Measurements', 'iv_coef':'Instrumental Variable'}
ape_names_mapper = {'mismeasured_coef_ape':'Single Measurement', 'mismeasured_allvar_coef_ape':'All Measurements', 'pca_ape':'PCA', 'mismeasured_avg_coef_ape':'Average of Measurements', 'iv_coef_ape':'Instrumental Variable'}

In [16]:
sim_reg_p = (output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')]
                   .drop(columns=['covariance', 'beta1', 'beta2'])
                   .groupby('p')
                   .agg(['mean', 'std'])
                   .transpose()
                   .reset_index()
                   .rename(columns = {5:'five', 20:'twenty', 50:'fifty'}))

sim_reg_p

p,level_0,level_1,five,twenty,fifty
0,mismeasured_coef,mean,1.239877,1.264090,1.299301
1,mismeasured_coef,std,NaN,NaN,NaN
2,mismeasured_allvar_coef,mean,1.130403,1.054896,1.043388
3,mismeasured_allvar_coef,std,NaN,NaN,NaN
4,mismeasured_avg_coef,mean,1.305751,1.296471,1.283434
...,...,...,...,...,...
17,mismeasured_allvar_coef_ape,std,NaN,NaN,NaN
18,mismeasured_avg_coef_ape,mean,0.305751,0.296471,0.283434
19,mismeasured_avg_coef_ape,std,NaN,NaN,NaN
20,iv_coef_ape,mean,0.172444,0.067577,0.056073


In [17]:
sim_reg_p_coefs = (sim_reg_p.query('level_0.str.contains("coef")')
                            .query('level_0 != "true_val_coef"')
                            .query('not level_0.str.contains("ape")')
                            .round(3)
                            .astype(str))

sim_reg_p_coefs['five'] = sim_reg_p_coefs.apply(lambda x: std_parenth_padder(x.level_1, x.five), axis = 1)
sim_reg_p_coefs['twenty'] = sim_reg_p_coefs.apply(lambda x: std_parenth_padder(x.level_1, x.twenty), axis = 1)
sim_reg_p_coefs['fifty'] = sim_reg_p_coefs.apply(lambda x: std_parenth_padder(x.level_1, x.fifty), axis = 1)

sim_reg_p_coefs['level_0'] = sim_reg_p_coefs.apply(lambda x: format_level_0(x.level_1, x.level_0), axis = 1)

sim_reg_p_coefs = (sim_reg_p_coefs.drop(columns = 'level_1')
                                  .replace(coef_names_mapper))

Minimal_Latex(sim_reg_p_coefs, tables_dir + '/p_coefs_exp.tex')

sim_reg_p_coefs

p,level_0,five,twenty,fifty
0,Single Measurement,1.24,1.264,1.299
1,,(nan),(nan),(nan)
2,All Measurements,1.13,1.055,1.043
3,,(nan),(nan),(nan)
4,Average of Measurements,1.306,1.296,1.283
5,,(nan),(nan),(nan)
6,PCA,1.145,1.084,1.068
7,,(nan),(nan),(nan)
10,Instrumental Variable,1.172,1.068,1.056
11,,(nan),(nan),(nan)


In [18]:
sim_reg_p_apes = (sim_reg_p.query('level_0.str.contains("ape")')
                           .query('level_1 != "std"')
                           .drop(columns = 'level_1')
                           .replace(ape_names_mapper)
                           .set_index('level_0')
                           .multiply(100)
                           .round(1)
                           .astype(str)
                           .add('%')
                           .reset_index())

Minimal_Latex(sim_reg_p_apes, tables_dir + '/p_apes_exp.tex')

sim_reg_p_apes

p,level_0,five,twenty,fifty
0,PCA,14.5%,8.4%,6.8%
1,Single Measurement,24.0%,26.4%,29.9%
2,All Measurements,13.0%,5.5%,4.3%
3,Average of Measurements,30.6%,29.6%,28.3%
4,Instrumental Variable,17.2%,6.8%,5.6%


In [19]:
output.loc[(output['p']==5)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,0.1,1.0,5,0.369439,0.259652,0.443448,0.266809,0.074569,0.314139,1.668085,2.694394,1.596524,3.434482,2.141391
1,0.5,1.0,1.0,5,1.239877,1.130403,1.305751,1.144760,0.956871,1.172444,0.144760,0.239877,0.130403,0.305751,0.172444
2,0.5,10.0,1.0,5,10.257260,10.148153,10.348358,10.168153,9.979385,10.205653,0.016815,0.025726,0.014815,0.034836,0.020565


In [20]:
output.loc[(output['p']==5)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,0.1,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.5,10.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
output.loc[(output['beta1']==1)&(output['p']==5)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,0.1,5,1.037412,1.031412,1.043361,1.033029,1.008602,1.040223,0.033029,0.037412,0.031412,0.043361,0.040223
1,0.5,1.0,1.0,5,1.239877,1.130403,1.305751,1.144760,0.956871,1.172444,0.144760,0.239877,0.130403,0.305751,0.172444
2,0.5,1.0,10.0,5,3.835763,2.736777,4.889809,3.033415,0.955749,3.386664,2.033415,2.835763,1.736777,3.889809,2.386664


In [22]:
output.loc[(output['beta1']==1)&(output['p']==5)&(output['covariance'] == 0.5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,0.1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.5,1.0,10.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['p'] == 5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,0.254457,0.394852,0.161883,0.328149,0.993651,0.322783,0.671851,0.745543,0.605148,0.838117,0.677217
1,-0.5,1.0,1.0,5,0.718394,0.822276,0.640263,0.806271,1.006469,0.769591,0.193729,0.281606,0.177724,0.359737,0.230409
2,0.0,1.0,1.0,5,1.015138,1.014898,1.025816,1.033058,1.018260,1.026033,0.033058,0.015138,0.014898,0.025816,0.026033
3,0.5,1.0,1.0,5,1.239877,1.130403,1.305751,1.144760,0.956871,1.172444,0.144760,0.239877,0.130403,0.305751,0.172444
4,0.9,1.0,1.0,5,1.779489,1.608337,1.825720,1.681415,0.894170,1.664195,0.681415,0.779489,0.608337,0.825720,0.664195


In [24]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['p'] == 5)&(output['exp_of_var']=='yes')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.9,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p','exp_of_var']).mean().reset_index()

,covariance,beta1,beta2,p,exp_of_var,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,1.0,5,no,1.259002,1.107635,1.103682,1.103888,0.988199,1.128105,0.103888,0.259002,0.107635,0.103682,0.128105
1,0.5,1.0,1.0,20,no,1.270695,1.029734,1.028652,1.028752,0.999400,1.036479,0.028752,0.270695,0.029734,0.028652,0.036479
2,0.5,1.0,1.0,50,no,1.309008,1.015198,1.015221,1.016186,1.006790,1.029574,0.016186,0.309008,0.015198,0.015221,0.029574


In [26]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p','exp_of_var']).std().reset_index()

,covariance,beta1,beta2,p,exp_of_var,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,1.0,5,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,1.0,1.0,20,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.5,1.0,1.0,50,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
output.loc[(output['p']==5)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,0.1,1.0,5,0.381021,0.218270,0.220204,0.219955,0.112098,0.247333,1.199553,2.810206,1.182697,1.202041,1.473329
1,0.5,1.0,1.0,5,1.259002,1.107635,1.103682,1.103888,0.988199,1.128105,0.103888,0.259002,0.107635,0.103682,0.128105
2,0.5,10.0,1.0,5,10.284815,10.123299,10.123861,10.123623,10.025852,10.141373,0.012362,0.028481,0.012330,0.012386,0.014137


In [28]:
output.loc[(output['p']==5)&(output['beta2']==1)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,0.1,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.5,10.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
output.loc[(output['beta1']==1)&(output['p']==5)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,0.1,5,1.047652,1.032653,1.029586,1.030001,1.020066,1.030305,0.030001,0.047652,0.032653,0.029586,0.030305
1,0.5,1.0,1.0,5,1.259002,1.107635,1.103682,1.103888,0.988199,1.128105,0.103888,0.259002,0.107635,0.103682,0.128105
2,0.5,1.0,10.0,5,3.875268,2.014999,2.015542,2.013079,0.994283,2.201372,1.013079,2.875268,1.014999,1.015542,1.201372


In [30]:
output.loc[(output['beta1']==1)&(output['p']==5)&(output['covariance'] == 0.5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,0.5,1.0,0.1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.5,1.0,10.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
sim_reg_rho = (output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['p'] == 5)&(output['exp_of_var']=='no')]
                   .drop(columns=['p', 'beta1', 'beta2'])
                   .groupby('covariance')
                   .agg(['mean', 'std'])
                   .transpose()
                   .reset_index())
                   #.rename(columns = {5:'five', 20:'twenty', 50:'fifty'}))

sim_reg_rho

covariance,level_0,level_1,-0.9,-0.5,0.0,0.5,0.9
0,mismeasured_coef,mean,0.296391,0.729930,1.021665,1.259002,1.753470
1,mismeasured_coef,std,NaN,NaN,NaN,NaN,NaN
2,mismeasured_allvar_coef,mean,0.539387,0.909727,1.004850,1.107635,1.513187
3,mismeasured_allvar_coef,std,NaN,NaN,NaN,NaN,NaN
4,mismeasured_avg_coef,mean,0.540122,0.910428,1.005135,1.103682,1.512977
...,...,...,...,...,...,...,...
17,mismeasured_allvar_coef_ape,std,NaN,NaN,NaN,NaN,NaN
18,mismeasured_avg_coef_ape,mean,0.459878,0.089572,0.005135,0.103682,0.512977
19,mismeasured_avg_coef_ape,std,NaN,NaN,NaN,NaN,NaN
20,iv_coef_ape,mean,0.515494,0.110197,0.003627,0.128105,0.565119


In [24]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['p'] == 5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,0.217245,0.477989,0.477303,0.477019,0.952746,0.422875,0.522981,0.782755,0.522011,0.522697,0.577125
1,-0.5,1.0,1.0,5,0.724498,0.920783,0.919703,0.918035,1.012534,0.904684,0.081965,0.275502,0.079217,0.080297,0.095316
2,0.0,1.0,1.0,5,1.005180,1.001407,1.000276,1.000394,1.009075,1.000825,0.000394,0.005180,0.001407,0.000276,0.000825
3,0.5,1.0,1.0,5,1.302670,1.121081,1.120034,1.119961,1.021782,1.141248,0.119961,0.302670,0.121081,0.120034,0.141248
4,0.9,1.0,1.0,5,1.813492,1.456942,1.457083,1.457555,1.000043,1.501181,0.457555,0.813492,0.456942,0.457083,0.501181


In [25]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['p'] == 5)&(output['exp_of_var']=='no')].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.5,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.9,1.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
